In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# all_events_data_w_time 

In [3]:
all_events = pd.read_csv('./cleaned_data/all_events_data_w_time.csv')

In [44]:
all_events.head()

,Unnamed: 0,EVENTS,SUBJECT_ID,TIME,EVE_INDEX
0,0,send 500mg vial,10,2103-06-28 00:00:00,4264
1,1,neo*iv*ampicillin sodium,10,2103-06-28 00:00:00,3505
2,2,991,10,2103-06-28 11:36:00,1064
3,3,966,10,2103-06-28 11:36:00,1041
4,4,V30,10,2103-06-28 11:36:00,1126


In [6]:
all_events[all_events.SUBJECT_ID == 70463 & (all_events.EVENTS == "428")]

,Unnamed: 0,EVENTS,SUBJECT_ID,TIME,EVE_INDEX


In [45]:
np.unique(all_events.SUBJECT_ID).shape

(46520,)

In [46]:
# event counts of all ids

all_count = all_events.groupby('SUBJECT_ID')['EVE_INDEX'].agg({'count':lambda x: len(x)})

In [47]:
all_count.reset_index(inplace=True)

In [48]:
all_count.head()

,SUBJECT_ID,count
0,2,8
1,3,15
2,4,77
3,5,4
4,6,176


In [49]:
all_count.describe()

,SUBJECT_ID,count
count,46520.000000,46520.000000
mean,34425.772872,116.710426
std,28330.400343,145.986396
min,2.000000,1.000000
25%,12286.750000,31.000000
50%,24650.500000,79.000000
75%,55477.500000,145.000000
max,99999.000000,3032.000000


In [50]:
# Control / Case group
case_id = pd.unique(all_events.ix[all_events['EVENTS'].str[:3] == '428','SUBJECT_ID'])
control_id =np.setdiff1d(np.unique(all_events['SUBJECT_ID'].values),case_id)

In [51]:
case_id.shape

(10209,)

In [52]:
control_id.shape

(36311,)

### filter events 

In [53]:
control = all_events.ix[np.in1d(all_events['SUBJECT_ID'],control_id),:]
case = all_events.ix[np.in1d(all_events['SUBJECT_ID'],case_id),:]

In [54]:
control.head()

,Unnamed: 0,EVENTS,SUBJECT_ID,TIME,EVE_INDEX
0,0,send 500mg vial,10,2103-06-28 00:00:00,4264
1,1,neo*iv*ampicillin sodium,10,2103-06-28 00:00:00,3505
2,2,991,10,2103-06-28 11:36:00,1064
3,3,966,10,2103-06-28 11:36:00,1041
4,4,V30,10,2103-06-28 11:36:00,1126


In [55]:
np.unique(control.SUBJECT_ID).shape

(36311,)

In [56]:
np.unique(case.SUBJECT_ID).shape

(10209,)

### index date control

In [57]:
observation_window = 2000

In [58]:
control_index_date = control.groupby(['SUBJECT_ID'], \
                    as_index=False)['TIME'].agg({'INDEX_DATE': lambda x: pd.to_datetime(x).max()})

In [ ]:
control_filter = pd.merge(control,control_index_date,how='left',on = ['SUBJECT_ID'])

In [ ]:
#observation window is set to be 2000 days
choice1 = pd.to_datetime(control_filter['TIME'])>= pd.to_datetime(control_filter['INDEX_DATE']) - pd.DateOffset(observation_window)
choice2 = pd.to_datetime(control_filter['TIME']) <= pd.to_datetime(control_filter['INDEX_DATE'])
control_filter = control_filter[choice1 & choice2]
control_filter.head()    

In [ ]:
# control count
count_control = control_filter.groupby('SUBJECT_ID').apply(lambda x: x.EVE_INDEX.size)
count_control.hist(bins=100)
print(np.histogram(count_control))
print(np.mean(count_control)); print(np.median(count_control)); print(np.std(count_control))

In [ ]:
# filter out control IDs with count >500 & count<30 ??
control_filter =pd.merge(control_filter, all_count,on='SUBJECT_ID',how='left')
control_filter_out = control_filter.ix[((control_filter['count']>30) & (control_filter['count']<500)),:]

In [ ]:
np.unique(control_filter_out.SUBJECT_ID).shape

In [ ]:
control_filter_out.head()

### index date case

In [ ]:
case_index_date = case.ix[case['EVENTS'].str[:3]=='428',:].groupby(['SUBJECT_ID'], \
                        as_index=False)['TIME'].agg({'INDEX_DATE': \
                        lambda x: pd.to_datetime(x).min()- pd.to_timedelta('90 days')})

In [ ]:
case_index_date.head()

In [ ]:
case_filter = pd.merge(case,case_index_date,how='left',on = ['SUBJECT_ID'])

In [ ]:
case_filter.head()

In [ ]:
choice1 = pd.to_datetime(case_filter['TIME'])>= pd.to_datetime(case_filter['INDEX_DATE']) - pd.DateOffset(2000)
choice2 = pd.to_datetime(case_filter['TIME']) <= pd.to_datetime(case_filter['INDEX_DATE'])
case_filter = case_filter[choice1 & choice2]
case_filter.head()    

In [ ]:
# case count
count_case = case_filter.groupby('SUBJECT_ID').apply(lambda x: x.EVE_INDEX.size)
count_case.hist(bins=100)
print(np.histogram(count_case))
print(np.mean(count_case)); print(np.median(count_case)); print(np.std(count_case))

count_case.describe()

In [ ]:
# filter out <10 & > 1000???

case_filter =pd.merge(case_filter, all_count,on='SUBJECT_ID',how='left')
case_filter_out = case_filter.ix[((case_filter['count']>30) & (case_filter['count']<500)),:]

In [ ]:
case_filter_out.head()

In [ ]:
np.unique(case_filter_out.SUBJECT_ID).size

In [ ]:
case_filter_out.shape

## output 

In [ ]:
# get sample from control
np.random.seed(6250)
sample_ids = np.random.choice(np.unique(control_filter_out.SUBJECT_ID),2*np.unique(case_filter_out.SUBJECT_ID).size,replace=False)

In [ ]:
control_out = control_filter_out.ix[np.in1d(control_filter_out['SUBJECT_ID'],sample_ids),:]

In [ ]:
np.unique(control_out.SUBJECT_ID).size

In [ ]:
control_out.reset_index(inplace=True)

In [ ]:
control_out.ix[:,['SUBJECT_ID','TIME','EVE_INDEX']].to_csv('./cleaned_data/control_w_time.csv')

In [ ]:
case_filter_out.ix[:,['SUBJECT_ID','TIME','EVE_INDEX']].to_csv('./cleaned_data/case_w_time.csv')